In [408]:
import numpy as np

def get_dummies(df, categoricalNumericFeatures=[]):
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]) or categoricalNumericFeatures.count(col) > 0:
            df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=True)
    return df

def bin_groups(df, categoricalNumericFeatures=[], percent=.05):
    import pandas as pd

    for col in df:
        # if the column is categorical
        if not pd.api.types.is_numeric_dtype(df[col]) or categoricalNumericFeatures.count(col) > 0:
            for group, count in df[col].value_counts().iteritems():
                # For each categorical value, if the percentage of the column that contains that value is below the threshold, relabel to other
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

def drop_cols_too_unique(df, percent = .80):

    for col in df:
        pct = len(df[df[col] == 'Other'][col]) / len(df)
        if pct >=.80:
            df.drop(columns=col, inplace=True)
            print("dropped " + col + ", " + str(pct * 100) + "% unique")
    return df        




class MissingData:

    def __init__(self, df, categoricalImputer='', numericImputer='', categoricalNumericFeatures=[], dropColPct = .5):
        self.categoricalImputer = categoricalImputer
        self.numericImputer = numericImputer
        self.df = df
        self.categoricalNumericFeatures = categoricalNumericFeatures
        self.dropColPct = dropColPct

    def fillMissingData(self):
        # get number of categorical and numeric columns missing data
        nCat, nNum = self.missing_values_check(self.df,self.categoricalNumericFeatures)

        # if no columns are missing data, we are done
        if nCat + nNum == 0:
            print("Missing data check complete. No columns have missing data.")
            return
        
        # drop columns missing too much data
        self.df = self.drop_cols_missing_data(self.df,self.dropColPct)

        # handle categorical columns missing data
        if nCat != 0:
            if self.categoricalImputer == 'most_frequent':
                imp = self.impute_mode()
            else:
                raise ValueError("Invalid categorical imputer.")
        
            self.df = self.selective_sklearn_impute(self.df, imp,'categorical',self.categoricalNumericFeatures)
            print(f"imputed {nCat} categorical column(s) using {self.categoricalImputer}.")
        
        # handle numeric columns missing data
        if nNum != 0: 
            if self.numericImputer == 'mean':
                imp = self.impute_mean()
            elif self.numericImputer == 'knn':
                imp = self.impute_KNN()
            elif self.numericImputer == 'reg':
                imp = self.impute_reg()
            else: 
                raise ValueError("Invalid numeric imputer.")
            
            self.df = self.selective_sklearn_impute(self.df,imp,'numeric',self.categoricalNumericFeatures)
            print(f"imputed {nNum} numeric column(s) using {self.numericImputer}.")

        # Check that filling in the missing data worked
        # get number of categorical and numeric columns missing data
        nCat, nNum = self.missing_values_check(self.df,self.categoricalNumericFeatures)

        # if no columns are missing data, we are done
        if nCat + nNum == 0:
            print("Missing data check complete. No columns have missing data.")
        else:
            raise ValueError(f"Missing data check failed. {nCat} categorical columns and {nNum} numeric columns have missing data.")
        return self.df

    def missing_values_report(self):
        df = self.df

        for col in df:
            print(f'{col}\t{round(df[col].isnull().sum() / len(df) * 100,2)}%')

    def missing_values_check(self, df, categoricalNumericFeatures):
        catColsMissingData = 0
        numColsMissingData = 0
        
        for col in df:
            #if there is at least one null value in the column
            if df[col].isnull().sum() > 0:

                #if its a categorical column
                if not pd.api.types.is_numeric_dtype(df[col]) or col in categoricalNumericFeatures:
                    catColsMissingData += 1
                else:
                    numColsMissingData +=1
        
        return catColsMissingData, numColsMissingData

    def drop_cols_missing_data(self, df, percent=.5):
        for col in df:
            if df[col].isna().sum()/len(df) > percent:
                df = df.drop(columns=col)
                print("dropped the " + col + " column. Over " + str(round(percent*100)) + "% of records were Null." )
        return df

    # IMPUTATION 
    def selective_sklearn_impute(self, df, imp, type, categoricalNumericFeatures = []):
        validTypes = ['categorical', 'numeric']
        if type not in validTypes:
            raise ValueError("Invalid type. Expected one of: %s" % validTypes)
        
        if type == 'categorical':
            # Only impute on categorical columns
            dfCategorical = df.select_dtypes(include='object')
            # if there are categorical features that are numerical, include them (ex: 1, 0 for survived)
            if len(categoricalNumericFeatures) != 0:
                for colName in categoricalNumericFeatures:
                    dfCategorical = dfCategorical.join(df[colName])
        
            dfCategorical = pd.DataFrame(imp.fit_transform(dfCategorical), columns=dfCategorical.columns, index=dfCategorical.index)

            # Replace the old categorical columns with the new imputed ones.
            df.drop(columns=dfCategorical.columns, inplace=True)
            df = df.join(dfCategorical)
        elif type == 'numeric':
            # Only impute on numeric columns
            dfNumeric = df.select_dtypes(include='number')
            # if there are categorical features that are numerical, don't include them (ex: 1, 0 for survived)
            if len(categoricalNumericFeatures) != 0:
                for colName in categoricalNumericFeatures:
                    if colName in dfNumeric.columns:
                        dfNumeric = dfNumeric.drop(columns=colName)
            dfNumeric = pd.DataFrame(imp.fit_transform(dfNumeric), columns=dfNumeric.columns, index=dfNumeric.index)

            # Replace the old numeric columns with the new imputed ones.
            df.drop(columns=dfNumeric.columns, inplace=True)
            df = df.join(dfNumeric)
        return df

    ## CATEGORICAL IMPUTATION
    def impute_mode(self):
        from sklearn.impute import SimpleImputer

        imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

        return imp

    ## NUMERICAL IMPUTATION
    def impute_mean(self):
        from sklearn.impute import SimpleImputer

        # Change the strategy to mean, median, or mode
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')

        return imp

    def impute_KNN(self):
        from sklearn.impute import KNNImputer
        from sklearn.preprocessing import MinMaxScaler

        # Clustering is biased by unstandardized data; so MinMax scale it
        #df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns = df.columns)

        print("did you standardize the data before impute KNN?")
        imp = KNNImputer(n_neighbors=5, weights="uniform")
        return imp

    def impute_reg(self):
        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer

        # Scaling is unnecessary for regression-based imputation
        imp = IterativeImputer(max_iter=10, random_state=12345)
        return imp

  

In [409]:
import pandas as pd

df_train = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\train.csv")
df_test = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\test.csv")

df_train.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)

df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [410]:
df = df_train.copy()

categoricalNumericFeatures = ['Pclass','Survived'] # pclass is numeric but actually is a categorical feature
missingData = MissingData(df, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.8)
missingData.missing_values_report()

Survived	0.0%
Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	19.87%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.0%
Cabin	77.1%
Embarked	0.22%


### Cabin and Embarked are categorical columns, and right now I will only be using mode imputation for categorical columns. Since imputing mode on cabin will likely not yield any great results, I will drop cabin and continue with imputation

In [411]:
df.drop(columns='Cabin', inplace=True)
df = missingData.fillMissingData()

imputed 1 categorical column(s) using most_frequent.
imputed 1 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [412]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.

df = bin_groups(df,categoricalNumericFeatures, percent=.05)
df = drop_cols_too_unique(df,percent=.8)
df.head()

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


,Sex,Embarked,Pclass,Survived,Age,SibSp,Parch,Fare
PassengerId,,,,,,,,
1,male,S,3,0,22.0,1.0,0.0,7.2500
2,female,C,1,1,38.0,1.0,0.0,71.2833
3,female,S,3,1,26.0,0.0,0.0,7.9250
4,female,S,1,1,35.0,1.0,0.0,53.1000
5,male,S,3,0,35.0,0.0,0.0,8.0500


In [413]:
# dummy code
df = get_dummies(df,categoricalNumericFeatures)
df.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Survived_1
PassengerId,,,,,,,,,,
1,22.0,1.0,0.0,7.2500,1,0,1,0,1,0
2,38.0,1.0,0.0,71.2833,0,0,0,0,0,1
3,26.0,0.0,0.0,7.9250,0,0,1,0,1,1
4,35.0,1.0,0.0,53.1000,0,0,1,0,0,1
5,35.0,0.0,0.0,8.0500,1,0,1,0,1,0


In [414]:
def fit_mlr(df, test_size=.2, random_state=12345, label=''):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    
    X = df.drop(label,axis=1)
    y = df[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression().fit(X_train, y_train)
    print(f'R-squared (mlr): \t{model.score(X_test, y_test)}')

    return model

def fit_crossvalidate_mlr(df, k, label, repeat=True):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    from numpy import mean, std

    X = df.drop(label,axis=1)
    y = df[label]

    # prepare the cross-validation procedure
    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)
    

    # evaluate model
    scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)

    # report performance
    print(f'R-squared scores: \n{scores}\n')
    print(f'Average R-squared:\t{mean(scores)}')

    return LinearRegression().fit(X, y)


In [415]:
from sklearn import preprocessing



dfMinMax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df),columns=df.columns,index=df.index)

#dfMinMax.hist(figsize=(15,10))

In [416]:
X = dfMinMax.drop(columns='Survived_1')
y = dfMinMax['Survived_1']

In [417]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from numpy import mean


cv = KFold(n_splits=10, random_state=12345, shuffle=True)

model_log = LogisticRegression(max_iter=100)
R2 =  mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
print(R2)



0.8024843945068664


In [418]:
model_log.fit(X,y)

LogisticRegression()

In [419]:
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [420]:
missingData = MissingData(df_test, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.8)
missingData.missing_values_report()

Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	20.57%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.24%
Cabin	78.23%
Embarked	0.0%


In [421]:
df_test.drop(columns='Cabin', inplace=True)
df_test = missingData.fillMissingData()

imputed 2 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [422]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.

df_test = bin_groups(df_test,categoricalNumericFeatures, percent=.05)
df_test = drop_cols_too_unique(df_test,percent=.8)
df_test.head()

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
892,3,male,Q,34.5,0.0,0.0,7.8292
893,3,female,S,47.0,1.0,0.0,7.0000
894,2,male,Q,62.0,0.0,0.0,9.6875
895,3,male,S,27.0,0.0,0.0,8.6625
896,3,female,S,22.0,1.0,1.0,12.2875


In [423]:
df_test = get_dummies(df_test,categoricalNumericFeatures)
df_test.head()

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,34.5,0.0,0.0,7.8292,0,1,1,1,0
893,47.0,1.0,0.0,7.0000,0,1,0,0,1
894,62.0,0.0,0.0,9.6875,1,0,1,1,0
895,27.0,0.0,0.0,8.6625,0,1,1,0,1
896,22.0,1.0,1.0,12.2875,0,1,0,0,1


In [424]:
dfMinMax2 = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_test),columns=df_test.columns,index=df_test.index)
dfMinMax2.head()

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,0.452723,0.000,0.000000,0.015282,0.0,1.0,1.0,1.0,0.0
893,0.617566,0.125,0.000000,0.013663,0.0,1.0,0.0,0.0,1.0
894,0.815377,0.000,0.000000,0.018909,1.0,0.0,1.0,1.0,0.0
895,0.353818,0.000,0.000000,0.016908,0.0,1.0,1.0,0.0,1.0
896,0.287881,0.125,0.111111,0.023984,0.0,1.0,0.0,0.0,1.0


In [425]:
dfMinMax2 = dfMinMax2[dfMinMax.drop(columns='Survived_1').columns]
prediction = {'Survived':model_log.predict(dfMinMax2)}
dfPredict = pd.DataFrame(prediction,index=dfMinMax2.index,dtype=int)


In [426]:
dfPredict.dtypes

Survived    int32
dtype: object

In [427]:
dfPredict.to_csv('predictions.csv')
dfPredict.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
